## Biblioteki

In [66]:
import json
import os
import requests
from bs4 import BeautifulSoup

In [72]:
def extract(ancestor, selector = None, attribute = None, returns_list= False):
    if selector: 
        if returns_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.get_text().strip() for tag in ancestor.select(selector)]
        if attribute: #jeżeli prawda wchodzimy do środka, jeżeli nie to nie potrzebujemy try except
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None 
        try:
            return opinion.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

In [73]:
selectors = {
                "opinion_id": ( None,'data-entry-id'),
                "author": ("span.user-post__author-name",),
                "recommendation": ("span.user-post__author-recomendation > em",),
                "content": ( "div.user-post__text",),
                "pros" : ( "div.review-feature__title--positives ~ div.review-feature__item", None, True),
                "cons" : ( "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
                "useful": ( 'span[id^="vote-yes"]',),
                "useless": ('span[id^="vote-no"]',),
                "post_date": ('span.user-post__published > time:nth-child(1)', "datetime"),
                "purchase_date": ('span.user-post__published > time:nth-child(2)', "datetime")
            }

In [69]:
product_id = "114700014"

#url = f"https://www.ceneo.pl/114700014#tab=reviews" #https://www.ceneo.pl/114700014
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"# /opinie-1


# Ekstrakcja składowych pojedynczych opinii
| Składowa | Selektor | Zmienna |
|----------|----------|---------|
| id opinii| ["data-entry-id"] | opinion_id|
| autor    | span.user-post__author-name |author |
| rekomendacja| span.user-post__author-recomendation > em |recommendation|
| gwiazdki | span.user-post__score-count| rating|
| treść    | div.user-post__text|content | 
| lista zalet |div.review-feature__title review-feature__title--positives | pros|
| lista wad | div.review-feature__title review-feature__title--negatives| cons|
| dla ilu przydatna | span[id^="vote-yes"]| useful|
| dla ilu nieprzydatna| span[id^="vote-no"]| useless|
| data wystawienia | span.user-post__published > time:nth-child(1)["datetime"]| post_date|
|data zakupu | span.user-post__published > time:nth-child(2)["datetime"] | purchase_rate |

In [74]:


single_opinion = {
"opinion_id" : opinion['data-entry-id'],
"author": opinion.select_one("span.user-post__author-name").get_text().strip(),## wyciągnie zawartość znacznikow rekurencyjne #.text jako alternatywa do get_text() 
#.strip() pozbywamy się białych znaków
"recommendation" : opinion.select_one('span.user-post__author-recomendation > em'),
"rating" : opinion.select_one("span.user-post__score-count"),
"content" : opinion.select_one("div.user-post__text").get_text().strip(),
"pros" : extract(opinion, "div.review-feature__title--positives ~ div.review-feature__item", None, True),
"cons" : extract(opinion, "div.review-feature__title--negatives ~ div.review-feature__item", None, True),
"useful" : opinion.select_one('span[id^="vote-yes"]'),
"useless" : opinion.select_one('span[id^="vote-no"]'),
"post_date" : opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"].strip(),
"purchase_date" : opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"].strip(),
}



In [75]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
            single_opinion = {
                key: extract(opinion,*value)
                    for key, value in selectors.items()
            }
            all_opinions.append(single_opinion)
       
    try:
        url = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None

https://www.ceneo.pl/114700014#tab=reviews
https://www.ceneo.pl/114700014/opinie-2
https://www.ceneo.pl/114700014/opinie-3
https://www.ceneo.pl/114700014/opinie-4
https://www.ceneo.pl/114700014/opinie-5
https://www.ceneo.pl/114700014/opinie-6
https://www.ceneo.pl/114700014/opinie-7
https://www.ceneo.pl/114700014/opinie-8
https://www.ceneo.pl/114700014/opinie-9
https://www.ceneo.pl/114700014/opinie-10
https://www.ceneo.pl/114700014/opinie-11
https://www.ceneo.pl/114700014/opinie-12
https://www.ceneo.pl/114700014/opinie-13
https://www.ceneo.pl/114700014/opinie-14
https://www.ceneo.pl/114700014/opinie-15
https://www.ceneo.pl/114700014/opinie-16
https://www.ceneo.pl/114700014/opinie-17
https://www.ceneo.pl/114700014/opinie-18


In [76]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii = False) #narodowe znaki diakrytyczne będą się poprawnie wyświetlać

In [ ]:
len(opinions)